In [43]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
# df[['sex', 'smoker', 'region']] = ord.fit_transform(df[['sex', 'smoker', 'region']])
def get_data(path):
    
    ord = OrdinalEncoder()
    data = pd.read_csv(path)
    b = ['age', 'sex', 'bmi', 'children', 'charges', 'region', 'smoker']
    data = data[b]
    data[['smoker']] = ord.fit_transform(data[['smoker']])
    x, y = data.values[:,:-1], data.values[:,-1]
    
    x_train, x_test, y_train, y_test= train_test_split(x,y, test_size=0.3, random_state= 42)
    
    c_transformer = ColumnTransformer([("ordinal",OrdinalEncoder(handle_unknown= 'use_encoded_value', unknown_value=-1),[1,5]),('non_transformed','passthrough',[0,2,3])])
    
    x_train = c_transformer.fit_transform(x_train)
    x_test = c_transformer.transform(x_test)
    
    # Scaling is not mandatory for TREE ALGOs. BUT BEST PRACTICE
    scaler = StandardScaler()
    x_train_scaled = scaler.fit_transform(x_train)
    x_test_scaled = scaler.transform(x_test)
    
    return x_train_scaled, x_test_scaled, y_train, y_test, c_transformer, scaler 
 

In [40]:
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor

def train_model():
    
    x_train, x_test, y_train, y_test, c_transformer, scaler = get_data("insurance.csv")
    
    xgb_r = XGBRegressor()
    gboosting_r = GradientBoostingRegressor(learning_rate=0.01, n_estimators=50)
    dtree_r = DecisionTreeRegressor(random_state=42, max_depth=4 )
    
    xgb_r.fit(x_train,y_train)
    gboosting_r.fit(x_train, y_train)
    dtree_r.fit(x_train, y_train)
    
    return xgb_r , gboosting_r, dtree_r , c_transformer, scaler

train_model()

(XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
              validate_parameters=1, verbosity=None),
 GradientBoostingRegressor(learning_rate=0.01, n_estimators=50),
 DecisionTreeRegressor(max_depth=4, random_state=42),
 ColumnTransformer(transformers=[('ordinal',
                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                 unknown_value=-1),
                                  [1, 5]),
 

In [42]:
import pandas as pd
import numpy as np
model = train_model()

while True:

    age = 25#int(input("How old are you? \n"))
    sex = 'm'#str(input("Which is your sex? If you are a male answer 'm', if you are a female answer with an 'f': \n"))
    if sex.lower() == 'm':
        sex = 'male'
    else:
        sex = 'female'
    bmi = 23 #float(input("Which is your bmi ('Body mass index')? \n"))
    child = 1 #int(input("How many children do you have? \n"))
    charge = 23452 #str(input("Do you smoke? Options: yes,no \n"))
    region = 'northeast' #str(input("Which is your residential area in the US? Options: northeast, southeast, southwest, northwest \n"))

    #Preprocess
    ct = model[-2]
    scaler = model[-1] #b = ['age', 'sex', 'bmi', 'children', 'charges', 'region', 'smoker']
    x = pd.DataFrame({"age":age, "sex":sex, "bmi":bmi, "child":child, "charges":charge," region":region}, index=[0])
    x_trans = ct.transform(x)
    x_scaled = scaler.transform(x)

    #Predict

    predictions = np.array([clf.predict(x_scaled) for clf in model[:-2]])
    print(predictions)
    print(f"Prediction for your conditions: {int(predictions.mean())}")

ValueError: could not convert string to float: 'male'